# Covid-19 Dashboard 

Install required packages in a conda environment

`conda install -c conda-forge ipywidgets voila widgetsnbextension voila-vuetify bqplot ipyvuetify`

Run with

`voila --debug main.ipynb`

In [243]:
# Required packages
import pandas as pd # For easy-to-use data structures and data analysis
import seaborn as sns # Advanced visualizations
import matplotlib.pyplot as mplt # For advanced visualization functions, which are not wrapped by seaborn
import matplotlib.dates as mdates # For date-time functions, which will be used also in visualizations
from ipywidgets import widgets, interact, interactive, fixed, interact_manual
import ipyvuetify as v
from bqplot import pyplot as plt
import bqplot

In [245]:
# Load dataset
df = pd.read_excel('data/final_tidy_data.xlsx')

In [294]:
df4 = df.reset_index()
#tmp_df_h4[(tmp_df_h4.YearWeek > '2022-01') & (tmp_df_h4.YearWeek < '2022-10')]
# Have a look at the number of Covid cases over whole time per federal state.
def plt_federal_state_over_time(yw_range):
    if len(yw_range) == 0:
        yw_range = (min(df4.YearWeek), max(df4.YearWeek))
    fig = mplt.figure()
    df = df4[(df4.YearWeek > yw_range[0]) & (df4.YearWeek < yw_range[1])]
    fg = sns.lineplot(
        data = df,
        x = 'YearWeek',
        y = 'NrCovidCases',
        hue = "FederalState",
        ci = None
    )
    fg.set_title('Covid cases over whole time per federal state')
    fg.xaxis.set_major_locator(mdates.DayLocator(interval=4))
    fig.add_axes(fg)
    # Rotate the x-axis labels
    mplt.xticks(rotation=90) 
    mplt.show()

def get_year_week_slider():
    options = df4['YearWeek'].unique()
    return widgets.SelectionRangeSlider(
        options=options,
        index=(0, len(options)-1),
        description='YearWeek',
        disabled=False
    )
interactive_plt_df4 = interactive(plt_federal_state_over_time, yw_range = get_year_week_slider())
output = interactive_plt_df4.children[-1]
output.layout.height = '530px'

#plt_federal_state_over_time(('2021-01', '2022-40'))

In [295]:
# Show the absolute numbers of whole dataset.
# Here we see the first male age group (0-64) has got much more deaths than females, with a compareable number of covid cases.
# The second age group (> 64) has got a slightly different number of Corona cases.
# However we see that that males have got lower number of Corona cases, but a higher number of death in contrast to females!
df1 = df.groupby(['Gender', 'AgeGroup'])[['NrCovidCases', 'NrCovidRecovered', 'NrCovidDeath']].sum()
#tmp_df_h1
#tmp_df[['NrCovidCases', 'NrCovidRecovered', 'NrCovidDeath']].sum()

def plt_abs_covid_cases():
    fig = mplt.figure()
    ax = df1[['NrCovidCases', 'NrCovidRecovered', 'NrCovidDeath']].plot.bar(title = "Absolute numbers of covid cases grouped by gender and age group")
    #mplt.ylabel("Absolute number")
    mplt.show()

interactive_plt_df1 = interactive(plt_abs_covid_cases, yw_range = get_year_week_slider())
output = interactive_plt_df1.children[-1]
output.layout.height = '530px'

#display(plt_abs_covid_cases())
#plt_abs_covid_cases()

In [296]:
def get_gender_radio_buttons():
    return widgets.RadioButtons(
        options = ['Female', 'Male'],
        value = 'Female',
        description = 'Gender:',
        disabled = False
    )

def get_age_radio_buttons():
    return widgets.RadioButtons(
        options = ['0-64', '>64'],
        value = '0-64',
        description=  'Age:',
        disabled = False
    )

def df1_data_table(gender, ageGroup):
    gender_mapping = {'Female': 'W', 'Male': 'M'}
    df_tmp = df1[df1.index.get_level_values('Gender').isin([gender_mapping[gender]]) & df1.index.get_level_values('AgeGroup').isin([ageGroup])]
    dt = v.DataTable()
    dt.headers = [
        {'text': 'NrCovidCases', 'value': 'NrCovidCases'},
        {'text': 'NrCovidRecovered', 'value': 'NrCovidRecovered'},
        {'text': 'NrCovidDeath', 'value': 'NrCovidDeath'},
    ]
    dt.items = df_tmp.to_dict(orient = 'records')
    display(dt)

gender_radio_buttons = get_gender_radio_buttons()
age_radio_buttons = get_age_radio_buttons()
interactive_df1_ui = widgets.HBox([gender_radio_buttons, age_radio_buttons])
interactive_df1 = widgets.interactive_output(df1_data_table, {'gender': gender_radio_buttons, 'ageGroup': age_radio_buttons})

ipywidgets.widgets.widget_box.Box

In [297]:
# Layout @see https://vuetifyjs.com/en/api/v-layout/

v.Tabs(children=[
    v.Tab(children=['Plots']),
    v.Tab(children=['Data Tables']),
    v.TabItem(children=[
        v.Layout(row=True, wrap=True, align_center=True, children=[
            v.Flex(xs12=True, lg6=True, xl6=True, children=[
                interactive_plt_df4
            ]),
            v.Flex(xs12=True, lg6=True, xl6=True, children=[
                interactive_plt_df1
            ]),
        ])
    ]),
    v.TabItem(children=[
        v.Layout(row=True, wrap=True, align_center=True, children=[
            v.Flex(xs12=True, lg6=True, xl6=True, children=[
                interactive_df1_ui, interactive_df1
            ]),
        ])
    ]),
])

Tabs(children=[Tab(children=['Plots']), Tab(children=['Data Tables']), TabItem(children=[Layout(align_center=T…

# Fast API

Calculate sum of...

In [ ]:
response = ''
try:
    response = requests.get('http://127.0.0.1:8000/calc/sum?a=1&b=2')
except Exception as e:
    print("API not available")

if response:
    data = response.json()
    print('Result: ' + str(data['result']))